Imports

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import helpers.evaluate
from helpers.model_list import model_map
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import warnings
from sklearn.exceptions import FitFailedWarning

#Import models
models = {name: model_class() for name, model_class in model_map.items()}

Load in Data

In [2]:
df = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
X = df.drop(columns=["Diabetes_binary"])
y = df["Diabetes_binary"]
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [ ]:
# Univariate Analysis
plt.figure()
df['BMI'].hist(bins=20)
plt.title("BMI Distribution")
plt.xlabel("BMI")
plt.ylabel("Frequency")
plt.show()

# 13-level age category 1 = 18-24 9 = 60-64 13 = 80 or older
plt.figure()
df['Age'].hist(bins=20)
plt.title("Age Distribution")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# Identify categorical/binary features to visualize their distributions.

categorical_features = [
    'Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck',
    'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity',
    'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare',
    'NoDocbcCost', 'Sex'
]

for col in categorical_features:
    plt.figure()
    df[col].value_counts().plot(kind='bar')
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.show()

In [ ]:
# Mulitvariate analysis

corr_matrix = df.corr()

plt.figure(figsize=(10, 8))
plt.matshow(corr_matrix, fignum=1)
plt.title("Correlation Matrix", pad=20)
plt.colorbar()
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)
plt.show()

In [ ]:
# 2. Scatter Plot: BMI vs Age 
plt.figure()
plt.scatter(df['BMI'], df['Age'], alpha=0.6)
plt.title("Scatter Plot: BMI vs Age")
plt.xlabel("BMI")
plt.ylabel("Age")
plt.show()

In [ ]:
diabetes_df=df[(df['Diabetes_binary']==1)]

for col in categorical_features:
    if(col!='Diabetes_binary'):
        plt.figure()
        diabetes_df[col].value_counts().plot(kind='bar')
        plt.title(f"Distribution of {col} with diabetes")
        plt.xlabel(col)
        plt.ylabel("Count")
        plt.show()

Explore Data

In [ ]:
print("Dataset shape:", df.shape)
print("\nData types:")
print(df.dtypes)


In [ ]:
print("\nSummary statistics:")
print(df.describe())

In [ ]:
# Count missing values in each column
print("\nMissing values per column:")
print(df.isnull().sum())

In [ ]:
# Check for duplicate records
print("\nNumber of duplicate rows:", df.duplicated().sum())

Clean Data

In [3]:
print("Dataset shape:", df.shape)
df = df.drop_duplicates()
print("New shape:", df.shape)

Dataset shape: (70692, 22)
New shape: (69057, 22)


Split Data

In [3]:
#Transform target
le = LabelEncoder()
y = le.fit_transform(y)

#Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

#Scale Data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Reassign col names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)


Train Models

In [ ]:


total_metrics = {}

for name, model in models.items():
    print(f"\n--- Training and Evaluating {name} ---")
    model.train(X_train_scaled, y_train)
    
    predictions = model.predict(X_test_scaled)
    
    metrics = helpers.evaluate.evaluate_model(y_test, predictions, name)
    total_metrics[name] = metrics


Report Stats

In [ ]:
# Get best metrics
bestModel = ""
bestPerformance = 0.0
for name, metrics in total_metrics.items():
    if metrics["Accuracy"] > bestPerformance:
        bestModel = name
        bestPerformance = metrics["Accuracy"]

print("Best Model =", bestModel)
print("Model Accuracy =", bestPerformance)


Hyperparameter Tuning

In [ ]:
#ignoring warnings when tuning hyperparams
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
for name, model in models.items():
    print(f"\n--- Tuning Hyperparameters for {name} ---")
    model.reset(name)
    model.tune_params(X_train_scaled, y_train, 10, name)

[I 2025-04-23 05:22:06,236] A new study created in RDB with name: Decision Tree



--- Tuning Hyperparameters for Decision Tree ---


[I 2025-04-23 05:22:06,512] Trial 0 finished with value: 0.7198327253514678 and parameters: {'criterion': 'log_loss', 'splitter': 'best', 'max_depth': 39, 'min_samples_split': 16, 'min_samples_leaf': 17, 'max_features': None}. Best is trial 0 with value: 0.7198327253514678.
[I 2025-04-23 05:22:06,797] Trial 1 finished with value: 0.6718774406623542 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 29, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 0.7198327253514678.
[I 2025-04-23 05:22:07,044] Trial 2 finished with value: 0.708161975174727 and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 12, 'max_features': None}. Best is trial 0 with value: 0.7198327253514678.
[I 2025-04-23 05:22:07,177] Trial 3 finished with value: 0.7166492886086829 and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 42, 'min_samples_split': 19, 'min_sa


--- Tuning Hyperparameters for Logistic Regression ---


[I 2025-04-23 05:22:08,450] Trial 0 pruned. 
[I 2025-04-23 05:22:08,521] Trial 1 pruned. 
[I 2025-04-23 05:22:08,587] Trial 2 pruned. 


...

[I 2025-04-23 05:22:35,191] Trial 0 finished with value: 0.718842126013602 and parameters: {'n_neighbors': 22, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 23, 'p': 5}. Best is trial 0 with value: 0.718842126013602.
[I 2025-04-23 05:22:36,554] Trial 1 finished with value: 0.7223791195128406 and parameters: {'n_neighbors': 16, 'weights': 'uniform', 'algorithm': 'brute', 'leaf_size': 11, 'p': 1}. Best is trial 1 with value: 0.7223791195128406.
[I 2025-04-23 05:23:29,680] Trial 2 finished with value: 0.7206813316128723 and parameters: {'n_neighbors': 24, 'weights': 'distance', 'algorithm': 'ball_tree', 'leaf_size': 57, 'p': 4}. Best is trial 1 with value: 0.7223791195128406.
[I 2025-04-23 05:23:32,514] Trial 3 finished with value: 0.7037771259060314 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 51, 'p': 1}. Best is trial 1 with value: 0.7223791195128406.
[I 2025-04-23 05:23:34,071] Trial 4 finished with value: 0.7275421538818


--- Tuning Hyperparameters for SVC ---


[I 2025-04-23 05:25:36,116] Trial 0 finished with value: 0.740839795485941 and parameters: {'kernel': 'sigmoid', 'degree': 18, 'gamma': 'auto', 'probability': False}. Best is trial 0 with value: 0.740839795485941.
[I 2025-04-23 05:26:02,586] Trial 1 finished with value: 0.7439523857888846 and parameters: {'kernel': 'rbf', 'degree': 15, 'gamma': 'auto', 'probability': False}. Best is trial 1 with value: 0.7439523857888846.
[I 2025-04-23 05:26:19,621] Trial 2 finished with value: 0.746639772370789 and parameters: {'kernel': 'linear', 'degree': 8, 'gamma': 'auto', 'probability': False}. Best is trial 2 with value: 0.746639772370789.
[I 2025-04-23 05:26:36,595] Trial 3 finished with value: 0.746639772370789 and parameters: {'kernel': 'linear', 'degree': 10, 'gamma': 'auto', 'probability': False}. Best is trial 2 with value: 0.746639772370789.
[I 2025-04-23 05:26:53,545] Trial 4 finished with value: 0.746639772370789 and parameters: {'kernel': 'linear', 'degree': 10, 'gamma': 'auto', 'proba

Skipping due to error


[I 2025-04-23 05:28:51,473] Trial 6 finished with value: 0.7422542976921372 and parameters: {'kernel': 'rbf', 'degree': 17, 'gamma': 'scale', 'probability': True}. Best is trial 2 with value: 0.746639772370789.
